In [1]:
from datasets import load_dataset
from setfit import SetFitModel
from train.active_learning import active_train
from train.train_config import TrainConfig
from train.active_learning_config import ActiveLearningConfig
from data.dataset_config import DatasetConfig

c:\Users\Colin\Documents\aml\aml_final\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("sst2")
train_config = TrainConfig(num_iterations=10)
active_learning_config = ActiveLearningConfig()
dataset_config = DatasetConfig()

In [3]:
def model_init():
    return SetFitModel.from_pretrained("thenlper/gte-small", use_differentiable_head=True, head_params={"out_features": dataset_config.num_classes},
)

In [4]:
trainer = active_train(model_init=model_init, dataset=dataset, train_config=train_config, active_learning_config=active_learning_config, dataset_config=dataset_config)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset
Generating Training Pairs: 100%|██████████| 10/10 [00:00<?, ?it/s]
***** Running training *****
  Num examples = 160
  Num epochs = 1
  Total optimization steps = 20
  Total train batch size = 8
Epoch: 100%|██████████| 1/1 [00:18<00:00, 18.95s/it]
Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.
Epoch: 100%|██████████| 10/10 [02:21<00:00, 14.18s/it]
Applying column mapping to evaluation dataset
***** Running evaluation *****


Accuracy: {'accuracy': 0.7419724770642202}
